# 数据规整
聚合、合并、重塑
# 层次化索引
层次化索引（hierarchical indexing）是pandas的一项重要功能，它使你能在一个轴上拥有多个（两个以上）索引级别。

In [4]:
import pandas as pd

In [6]:
xa=pd.Series(pd.np.random.randn(9),index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],[1, 2, 3, 1, 3, 1, 2, 2, 3]])
xa

a  1   -0.452423
   2    0.576528
   3    0.586482
b  1   -1.219562
   3   -0.881183
c  1    0.917401
   2    0.314518
d  2    0.292254
   3    0.212361
dtype: float64

In [7]:
xa.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

对于一个层次化索引的对象，可以使用所谓的部分索引，使用它选取数据子集的操作更简单：

In [8]:
xa['a']

1   -0.452423
2    0.576528
3    0.586482
dtype: float64

In [9]:
xa['a':'c']

a  1   -0.452423
   2    0.576528
   3    0.586482
b  1   -1.219562
   3   -0.881183
c  1    0.917401
   2    0.314518
dtype: float64

In [10]:
xa[['a','d']]

a  1   -0.452423
   2    0.576528
   3    0.586482
d  2    0.292254
   3    0.212361
dtype: float64

In [12]:
xa.loc[['a','d']]

a  1   -0.452423
   2    0.576528
   3    0.586482
d  2    0.292254
   3    0.212361
dtype: float64

甚至还可以在“内层”中进行选取

In [17]:
xa.loc[:,2]

a    0.576528
c    0.314518
d    0.292254
dtype: float64

层次化索引在数据重塑和基于分组的操作（如透视表生成）中扮演着重要的角色。
可以通过unstack方法将这段数据重新安排到一个DataFrame中

In [18]:
xa.unstack()

,1,2,3
a,-0.452423,0.576528,0.586482
b,-1.219562,NaN,-0.881183
c,0.917401,0.314518,NaN
d,NaN,0.292254,0.212361


unstack的逆运算是stack

In [19]:
xa.unstack().stack()

a  1   -0.452423
   2    0.576528
   3    0.586482
b  1   -1.219562
   3   -0.881183
c  1    0.917401
   2    0.314518
d  2    0.292254
   3    0.212361
dtype: float64

对于一个DataFrame，每条轴都可以有分层索引：

In [21]:
xa = pd.DataFrame(pd.np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])

In [22]:
xa

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [23]:
xa.index.names=['row1','row2']
xa.columns.names=['col1','col2']
xa

col1       Ohio     Colorado
col2      Green Red    Green
row1 row2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [29]:
xa['Ohio']

col2       Green  Red
row1 row2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [76]:
xa['Ohio','Green']

row1  row2
a     1       0
      2       3
b     1       6
      2       9
Name: (Ohio, Green), dtype: int32

In [77]:
xa.loc['a',2]

col1      col2 
Ohio      Green    3
          Red      4
Colorado  Green    5
Name: (a, 2), dtype: int32

# 重排与分级排序
有时，你需要重新调整某条轴上各级别的顺序，或根据指定级别上的值对数据进行排序。swaplevel接受两个级别编号或名称，并返回一个互换了级别的新对象（但数据不会发生变化）

In [71]:
xb=xa.swaplevel('row2','row1',axis='index')
xb

col1       Ohio     Colorado
col2      Green Red    Green
row2 row1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [72]:
xb.loc[1]

col1  Ohio     Colorado
col2 Green Red    Green
row1                   
a        0   1        2
b        6   7        8

In [74]:
xc=xa.swaplevel('col2','col1',axis='columns')
xc

col2      Green  Red    Green
col1       Ohio Ohio Colorado
row1 row2                    
a    1        0    1        2
     2        3    4        5
b    1        6    7        8
     2        9   10       11

In [78]:
xc['Green']

col1       Ohio  Colorado
row1 row2                
a    1        0         2
     2        3         5
b    1        6         8
     2        9        11

In [79]:
xa

col1       Ohio     Colorado
col2      Green Red    Green
row1 row2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [85]:
xa.sort_index(level=0)

col1       Ohio     Colorado
col2      Green Red    Green
row1 row2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [91]:
xa.sort_index(axis='columns',level=1,ascending=False)

col1      Ohio       Colorado
col2       Red Green    Green
row1 row2                    
a    1       1     0        2
     2       4     3        5
b    1       7     6        8
     2      10     9       11

# 根据级别汇总统计

In [92]:
xa

col1       Ohio     Colorado
col2      Green Red    Green
row1 row2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [93]:
xa.sum(axis=0,level=0)

col1  Ohio     Colorado
col2 Green Red    Green
row1                   
a        3   5        7
b       15  17       19

In [94]:
xa.sum(axis=1,level=1)

col2       Green  Red
row1 row2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

# 使用DataFrame的列进行索引

In [99]:
xa= pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
xa

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [101]:
xb=xa.set_index(['c','d'])
xb

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [102]:
xb.sum(level=0)

,a,b
c,,
one,3,18
two,18,10


In [103]:
xb.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# 合并数据集
pandas对象中的数据可以通过一些方式进行合并：
* pandas.merge可根据一个或多个键将不同DataFrame中的行连接起来。SQL或其他关系型数据库的用户对此应该会比较熟悉，因为它实现的就是数据库的join操作。
* pandas.concat可以沿着一条轴将多个对象堆叠到一起。
* 实例方法combine_first可以将重复数据拼接在一起，用一个对象中的值填充另一个对象中的缺失值。

# 数据库风格的DataFrame合并

In [115]:
xa=pd.DataFrame({'key1': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                'data1': range(7)})
xb=pd.DataFrame({'key2': ['a', 'b', 'd'],
                'data2': range(3)})

In [108]:
xa

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [110]:
xb

,key,data2
0,a,0
1,b,1
2,d,2


In [111]:
pd.merge(xa,xb)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [119]:
pd.merge(xb,xa,left_on='key2',right_on='key1',how='outer')

,key2,data2,key1,data1
0,a,0.0,a,2.0
1,a,0.0,a,4.0
2,a,0.0,a,5.0
3,b,1.0,b,0.0
4,b,1.0,b,1.0
5,b,1.0,b,6.0
6,d,2.0,NaN,NaN
7,NaN,NaN,c,3.0


In [120]:
xa=pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
              'data1': range(6)})
xb=pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
               'data2': range(5)})

In [121]:
xa

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [122]:
xb

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


多对多连接产生的是行的笛卡尔积

In [123]:
pd.merge(xa,xb,on='key',how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


要根据多个键进行合并，传入一个由列名组成的列表即可：

In [124]:
xa=pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                      'key2': ['one', 'two', 'one'],
                      'lval': [1, 2, 3]})

xb= pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

In [125]:
xa

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [126]:
xb

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [129]:
pd.merge(xa,xb,on=['key1','key2'],how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


重复列名的处理

In [132]:
pd.merge(xa,xb,on='key1',how='outer',suffixes=('_left','_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


# 索引上的合并

In [134]:
ya=xa.set_index('key1')
ya

,key2,lval
key1,,
foo,one,1
foo,two,2
bar,one,3


In [138]:
yb=xb.set_index('key1')
yb

,key2,rval
key1,,
foo,one,4
foo,one,5
bar,one,6
bar,two,7


In [139]:
pd.merge(ya,yb,left_index=True,right_index=True)

,key2_x,lval,key2_y,rval
key1,,,,
bar,one,3,one,6
bar,one,3,two,7
foo,one,1,one,4
foo,one,1,one,5
foo,two,2,one,4
foo,two,2,one,5


对于层次化索引的数据，事情就有点复杂了

In [140]:
ya=xa
ya

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [141]:
yb=xb.set_index(['key1','key2'])
yb

rval
key1 key2      
foo  one      4
     one      5
bar  one      6
     two      7

In [144]:
pd.merge(left=ya,right=yb,left_on=['key1','key2'],right_index=True,how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
0,foo,one,1.0,5.0
1,foo,two,2.0,NaN
2,bar,one,3.0,6.0
2,bar,two,NaN,7.0


DataFrame还有一个便捷的join实例方法，它能更为方便地实现按索引合并。它还可用于合并多个带有相同或相似索引的DataFrame对象，但要求没有重叠的列。

In [146]:
xa

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [154]:
xa.columns=list('abc')

In [155]:
xa

,a,b,c
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [156]:
xb=xa.copy()
xb.columns=list('def')
xc=xa.copy()
xc.columns=list('ghi')

In [159]:
xc

,g,h,i
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [162]:
xa.join([xb,xc])

,a,b,c,d,e,f,g,h,i
0,foo,one,1,foo,one,1,foo,one,1
1,foo,two,2,foo,two,2,foo,two,2
2,bar,one,3,bar,one,3,bar,one,3


# 轴向连接
数据合并运算也被称作连接（concatenation）、绑定（binding）或堆叠（stacking）

In [163]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [164]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

默认情况下，concat是在axis=0上工作的，最终产生一个新的Series。如果传入axis=1，则结果就会变成一个DataFrame（axis=1是列）

In [166]:
pd.concat([s1,s2,s3],axis=1,sort=False)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


这种情况下，另外的轴上没有重叠，从索引的有序并集（外连接）上就可以看出来

In [167]:
pd.concat([s1,s2,s3],axis=1,sort=False,join='inner')

,0,1,2


In [168]:
xa=pd.concat([s1,s2])

In [170]:
pd.concat([s1,xa],axis=1,sort=False,join='inner')

,0,1
a,0,0
b,1,1


你想要在连接轴上创建一个层次化索引。使用keys参数即可达到这个目的

In [171]:
pd.concat([s1,s2,s3],keys=list('123'))

1  a    0
   b    1
2  c    2
   d    3
   e    4
3  f    5
   g    6
dtype: int64

如果沿着axis=1对Series进行合并，则keys就会成为DataFrame的列头

In [172]:
pd.concat([s1,s2,s3],keys=list('123'),axis=1,sort=False)

,1,2,3
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


同样的逻辑也适用于DataFrame对象

In [174]:
xa = pd.DataFrame(pd.np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])

xb = pd.DataFrame(5 + pd.np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])


In [175]:
xa

,one,two
a,0,1
b,2,3
c,4,5


In [176]:
xb

,three,four
a,5,6
c,7,8


In [181]:
pd.concat([xa,xb],sort=False,axis=1,keys=['xa','xb'])

xa        xb     
  one two three four
a   0   1   5.0  6.0
b   2   3   NaN  NaN
c   4   5   7.0  8.0

# 合并重叠数据

In [184]:
import numpy as np
xa = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])

xb = pd.Series(np.arange(len(xa), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])

In [185]:
xa

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [186]:
xb

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [188]:
xa.combine_first(xb)

f    0.0
e    2.5
d    2.0
c    3.5
b    4.5
a    5.0
dtype: float64

In [189]:
xb.combine_first(xa)

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [190]:
ya = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})

yb = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

In [191]:
ya

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [192]:
yb

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [193]:
ya.combine_first(yb)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


In [195]:
yb.combine_first(ya)

,a,b,c
0,5.0,NaN,2.0
1,4.0,3.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


# 重塑和轴向旋转
有许多用于重新排列表格型数据的基础运算。这些函数也称作重塑（reshape）或轴向旋转（pivot）运算。
# 重塑层次化索引
层次化索引为DataFrame数据的重排任务提供了一种具有良好一致性的方式。主要功能有二：

stack：将数据的列“旋转”为行。
unstack：将数据的行“旋转”为列。

In [197]:
xa = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio','Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))

In [198]:
xa

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [201]:
xa.stack()

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [202]:
xa.stack().unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


默认情况下，unstack操作的是最内层（stack也是如此）。传入分层级别的编号或名称即可对其它级别进行unstack操作：

In [204]:
xa.stack().unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


# 将“长格式”旋转为“宽格式”
多个时间序列数据通常是以所谓的“长格式”（long）或“堆叠格式”（stacked）存储在数据库和CSV中的。我们先加载一些示例数据，做一些时间序列规整和数据清洗：

In [265]:
data = pd.read_csv('examples\macrodata.csv')

In [266]:
data.head(11)

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19
5,1960.0,2.0,2834.390,1792.9,298.152,460.400,1966.1,29.55,140.2,2.68,5.2,180.671,0.14,2.55
6,1960.0,3.0,2839.022,1785.8,296.375,474.676,1967.8,29.75,140.9,2.36,5.6,181.528,2.70,-0.34
7,1960.0,4.0,2802.616,1788.2,259.764,476.434,1966.6,29.84,141.1,2.29,6.3,182.287,1.21,1.08
8,1961.0,1.0,2819.264,1787.7,266.405,475.854,1984.5,29.81,142.1,2.37,6.8,182.992,-0.40,2.77
9,1961.0,2.0,2872.005,1814.3,286.246,480.328,2014.4,29.92,142.9,2.29,7.0,183.691,1.47,0.81


In [267]:
periods=pd.PeriodIndex(year=data.year,quarter=data.quarter,name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [268]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
columns

Index(['realgdp', 'infl', 'unemp'], dtype='object', name='item')

In [269]:
ldata = data.reindex(columns=columns)
ldata.head()

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2


In [270]:
ldata.index = periods.to_timestamp('D', 'end')
ldata.head()

item,realgdp,infl,unemp
date,,,
1959-03-31,2710.349,0.00,5.8
1959-06-30,2778.801,2.34,5.1
1959-09-30,2775.488,2.74,5.3
1959-12-31,2785.204,0.27,5.6
1960-03-31,2847.699,2.31,5.2


In [271]:
ldata=ldata.stack().reset_index().rename(columns={0:'value'})
ldata.tail()

,date,item,value
604,2009-06-30,infl,3.370
605,2009-06-30,unemp,9.200
606,2009-09-30,realgdp,12990.341
607,2009-09-30,infl,3.560
608,2009-09-30,unemp,9.600


**ldata**就是多个时间序列的长格式。<br>
关系型数据库（如MySQL）中的数据经常都是这样存储的，因为固定架构（即列名和数据类型）有一个好处：随着表中数据的添加，item列中的值的种类能够增加。在前面的例子中，date和item通常就是主键（用关系型数据库的说法），不仅提供了关系完整性，而且提供了更为简单的查询支持。

有的情况下，使用这样的数据会很麻烦，你可能会更喜欢DataFrame，不同的item值分别形成一列，date列中的时间戳则用作索引。DataFrame的pivot方法完全可以实现这个转换：<br>
前两个传递的值分别用作行和列索引，最后一个可选值则是用于填充DataFrame的数据列。

In [272]:
pivoded=ldata.pivot('date','item','value')
pivoded.tail()

item,infl,realgdp,unemp
date,,,
2008-09-30,-3.16,13324.600,6.0
2008-12-31,-8.79,13141.920,6.9
2009-03-31,0.94,12925.410,8.1
2009-06-30,3.37,12901.504,9.2
2009-09-30,3.56,12990.341,9.6


pivot其实就是用set_index创建层次化索引，再用unstack重塑

In [273]:
ldata.set_index(['date','item']).unstack('item').head()

value                
item        infl   realgdp unemp
date                            
1959-03-31  0.00  2710.349   5.8
1959-06-30  2.34  2778.801   5.1
1959-09-30  2.74  2775.488   5.3
1959-12-31  0.27  2785.204   5.6
1960-03-31  2.31  2847.699   5.2

# 将“宽格式”旋转为“长格式”
旋转DataFrame的逆运算是pandas.melt。它不是将一列转换到多个新的DataFrame，而是合并多个列成为一个，产生一个比输入长的DataFrame。

In [312]:
sdata=ldata.pivot('date','item','value').reset_index()
sdata.head()

item,date,infl,realgdp,unemp
0,1959-03-31,0.00,2710.349,5.8
1,1959-06-30,2.34,2778.801,5.1
2,1959-09-30,2.74,2775.488,5.3
3,1959-12-31,0.27,2785.204,5.6
4,1960-03-31,2.31,2847.699,5.2


In [313]:
sdata.melt(['date']).tail(11)

,date,item,value
598,2007-03-31,unemp,4.5
599,2007-06-30,unemp,4.5
600,2007-09-30,unemp,4.7
601,2007-12-31,unemp,4.8
602,2008-03-31,unemp,4.9
603,2008-06-30,unemp,5.4
604,2008-09-30,unemp,6.0
605,2008-12-31,unemp,6.9
606,2009-03-31,unemp,8.1
607,2009-06-30,unemp,9.2


In [315]:
sdata.melt(['date']).pivot('date','item','value').reset_index().head(11)

item,date,infl,realgdp,unemp
0,1959-03-31,0.00,2710.349,5.8
1,1959-06-30,2.34,2778.801,5.1
2,1959-09-30,2.74,2775.488,5.3
3,1959-12-31,0.27,2785.204,5.6
4,1960-03-31,2.31,2847.699,5.2
5,1960-06-30,0.14,2834.390,5.2
6,1960-09-30,2.70,2839.022,5.6
7,1960-12-31,1.21,2802.616,6.3
8,1961-03-31,-0.40,2819.264,6.8
9,1961-06-30,1.47,2872.005,7.0
